<a href="https://colab.research.google.com/github/MelMacLondon/ML/blob/main/Transformers_Tokens_as_Colours_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Comparing Trained LLM Tokenizers

Lets see how several tokenizers associated with different LLMs work and explore how each tokenizer approaches tokenization differently.

## Setup

We start with setting up the lab by installing the `transformers` library and ignoring the warnings.


<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code>

In [21]:
!pip install transformers>=4.46.1

In [22]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

## Tokenizing Text

In this section, we will tokenize the sentence "Hello World!" using the tokenizer of the [`bert-base-cased` model](https://huggingface.co/google-bert/bert-base-cased).

Let's import the `Autotokenizer` class, define the sentence to tokenize, and instantiate the tokenizer.

<p style="background-color:#fff1d7; padding:15px; "> <b>FYI: </b> The transformers library has a set of Auto classes, like AutoConfig, AutoModel, and AutoTokenizer. The Auto classes are designed to automatically do the job for you.</p>

In [23]:
from transformers import AutoTokenizer

In [40]:
# define the sentence to tokenize
sentence = "Hello world! This is a demo around tokenization, lets check out the word antidisestablishmentarianism"

In [41]:
# load the pretrained tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

You'll now apply the tokenizer to the sentence. The tokeziner splits the sentence into tokens and returns the IDs of each token.

In [42]:
# apply the tokenizer to the sentence and extract the token ids
token_ids = tokenizer(sentence).input_ids

In [43]:
print(token_ids)

[101, 8667, 1362, 106, 1188, 1110, 170, 11238, 1213, 22559, 2734, 117, 11446, 4031, 1149, 1103, 1937, 2848, 10396, 16144, 1830, 10550, 1880, 7968, 1863, 102]


To map each token ID to its corresponding token, you can use the `decode` method of the tokenizer.

In [44]:
for id in token_ids:
    print(tokenizer.decode(id))

[CLS]
Hello
world
!
This
is
a
demo
around
token
##ization
,
lets
check
out
the
word
anti
##dis
##esta
##b
##lish
##ment
##arian
##ism
[SEP]


# English Morphology

English allows us to build new words like Lego blocks (e.g., "green" -> "greenness"). Because the tokenizer uses sub-words, it can handle these made-up words by breaking them into familiar parts.

In [29]:
morphology_test = ["green", "greenness", "greenacity", "greenification", "greenify", "greenly", "greendot", "greenwich"]

print(f"{'Word':<20} | Tokens")
print("-" * 50)

for word in morphology_test:
    tokens = tokenizer.tokenize(word)
    print(f"{word:<20} | {tokens}")

Word                 | Tokens
--------------------------------------------------
green                | ['green']
greenness            | ['green', '##ness']
greenacity           | ['green', '##acity']
greenification       | ['green', '##ification']
greenify             | ['green', '##ify']
greenly              | ['green', '##ly']
greendot             | ['green', '##do', '##t']
greenwich            | ['green', '##wich']


## Visualizing Tokenization

In this section, you'll wrap the code of the previous section in the function `show_tokens`. The function takes in a text and the model name, and prints the vocabulary length of the tokenizer and a colored list of the tokens.

In [30]:
# A list of colors in RGB for representing the tokens
colors = [
    '102;194;165', '252;141;98', '141;160;203',
    '231;138;195', '166;216;84', '255;217;47'
]

def show_tokens(sentence: str, tokenizer_name: str):
    """ Show the tokens each separated by a different color """

    # Load the tokenizer and tokenize the input
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    token_ids = tokenizer(sentence).input_ids

    # Extract vocabulary length
    print(f"Vocab length: {len(tokenizer)}")

    # Print a colored list of tokens
    for idx, t in enumerate(token_ids):
        print(
            f'\x1b[0;30;48;2;{colors[idx % len(colors)]}m' +
            tokenizer.decode(t) +
            '\x1b[0m',
            end=' '
        )

Here's the text that you'll use to explore the different tokenization strategies of each model.

In [31]:
text = """
English and CAPITALIZATION
🎵 鸟
show_tokens False None elif == >= else: two tabs:"    " Three tabs: "       "
12.0*50=600
"""

You'll now again use the tokenizer of `bert-base-cased` and compare its tokenization strategy to that of `Xenova/gpt-4`

**bert-base-cased**

In [32]:
show_tokens(text, "bert-base-cased")

Vocab length: 28996
[CLS] English and CA ##PI ##TA ##L ##I ##Z ##AT ##ION [UNK] [UNK] show _ token ##s F ##als ##e None el ##if = = > = else : two ta ##bs : " " Three ta ##bs : " " 12 . 0 * 50 = 600 [SEP] 

**Optional - bert-base-uncased**

You can also try the uncased version of the bert model, and compare the vocab length and tokenization strategy of the two bert versions.

In [33]:
show_tokens(text, "bert-base-uncased")

Vocab length: 30522
[CLS] english and capital ##ization [UNK] [UNK] show _ token ##s false none eli ##f = = > = else : two tab ##s : " " three tab ##s : " " 12 . 0 * 50 = 600 [SEP] 

**GPT-4**

In [34]:
show_tokens(text, "Xenova/gpt-4")

Vocab length: 100263

 English  and  CAPITAL IZATION 
 � � �  � � � 
 show _tokens  False  None  elif  ==  >=  else :  two  tabs :"      "  Three  tabs :  "         "
 12 . 0 * 50 = 600 
 

### Optional Models to Explore

You can also explore the tokenization strategy of other models. The following is a suggested list. Make sure to consider the following features when you're doing your comparison:
- Vocabulary length
- Special tokens
- Tokenization of the tabs, special characters and special keywords

**gpt2**

In [35]:
show_tokens(text, "gpt2")

Vocab length: 50257

 English  and  CAP ITAL IZ ATION 
 � � �  � � � 
 show _ t ok ens  False  None  el if  ==  >=  else :  two  tabs :"        "  Three  tabs :  "              " 
 12 . 0 * 50 = 600 
 

**Flan-T5-small**

In [36]:
show_tokens(text, "google/flan-t5-small")

Vocab length: 32100
English and CA PI TAL IZ ATION  <unk>  <unk> show _ to ken s Fal s e None  e l if = = > = else : two tab s : " " Three tab s : " " 12. 0 * 50 = 600  </s> 

**Starcoder 2 - 15B**

In [37]:
show_tokens(text, "bigcode/starcoder2-15b")

Vocab length: 49152

 English  and  CAPITAL IZATION 
 � � �   � � 
 show _ tokens  False  None  elif  ==  >=  else :  two  tabs :"      "  Three  tabs :  "         " 
 1 2 . 0 * 5 0 = 6 0 0 
 

**Phi-3**

In [38]:
show_tokens(text, "microsoft/Phi-3-mini-4k-instruct")

Vocab length: 32011
 
 English and C AP IT AL IZ ATION 
 � � � �  � � � 
 show _ to kens False None elif == >= else : two tabs :"    " Three tabs : "       " 
 1 2 . 0 * 5 0 = 6 0 0 
 

**Qwen2 - Vision-Language Model**

In [39]:
show_tokens(text, "Qwen/Qwen2-VL-7B-Instruct")

Vocab length: 151657

 English  and  CAPITAL IZATION 
 🎵  � � � 
 show _tokens  False  None  elif  ==  >=  else :  two  tabs :"      "  Three  tabs :  "         "
 1 2 . 0 * 5 0 = 6 0 0 
 